# 아래 주소에서 실제 colab 실제 동작된 결과를 확인할 수 있습니다.

- https://colab.research.google.com/drive/1pwoe7OlsfOBdVxHHaKtJDWIpJzs7KhhD?usp=sharing


## 들여쓰기 등이 colab 기준으로 되어 있어서 현 notebook 코드에선 알맞게 되어 있지 않을 수도 있습니다.

In [ ]:
!pip install selenium
!pip install requests
!pip install pandas
!pip install numpy
!pip install lxml
!pip install beautifulsoup4
!pip install newspaper3k
!pip install tqdm

!apt-get update
!apt install chromium-chromedriver

In [ ]:
import os
import lxml
import requests as r
import datetime as d
import time
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from multiprocessing import Process


In [ ]:
def set_driver():
    #chrome option을 window가 없이도 돌아가도록 설정
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome('chromedriver', chrome_options=chrome_options)
    return driver


In [ ]:
search_list = ['bts', 'korea', 'ai']

In [ ]:
def show_data(href_list, title_list, date_list, content_list):
    print(
        pd.DataFrame({
            'href' : href_list,
            'title' : title_list,
            'date' : date_list,
            'content' : content_list
        }).head()
    )

In [ ]:
driver = set_driver()
# search list를 하나씩 돌면서
for q in search_list:
  href_list, title_list, date_list, content_list = [], [], [], []
  url = f"https://edition.cnn.com/search/?size=10&q={q}&page="
  # 아래 range를 변형해서 더 많은 데이터를 수집할 수 있음
  for i in range(1, 4):
    # 처음일 때와 아닐 때 url 값이 바뀌어서 flag를 이용해 변환
    if i == 1:
      url = url + "&page=" + str(i)
    else:
      # 첫 페이지가 아닐 때는 뒤에 &from= (10 * (current_page - 1)) 이 붙음
      url = url.split("&page=")[0]
      url = url + "&page=" + str(i) + f"&from={10 * (i-1)}"
    driver.get(url)
    page_source = driver.page_source
    print(url)
    soup = BeautifulSoup(page_source, 'lxml')
    div_list = soup.findAll("div", {"class":"cnn-search__result-contents"})
    for d in div_list:
      try:
        # 제목, 링크, 콘텐츠, 날짜 값 저장
        href = d.find("h3", {"class":"cnn-search__result-headline"}).find('a').attrs['href']
        title = d.find("h3", {"class":"cnn-search__result-headline"}).find('a').text.strip()
        date = d.find("div", {"class" : "cnn-search__result-publish-date"}).findAll('span')[1].text.strip()
        content = d.find("div", {"class":"cnn-search__result-body"}).text.strip()
        href_list.append(href)
        title_list.append(title)
        date_list.append(date)
        content_list.append(content)
      except:
        continue
    url = url[:-1]
    # 약간의 시간차를 둠
    time.sleep(2)
  show_data(href_list, title_list, date_list, content_list)
  print(f"{q} end")
# end of driver
driver.close()

# use multiprocess

In [ ]:
def crawling(q, driver):
  href_list, title_list, date_list, content_list = [], [], [], []
  driver = set_driver()
  url = f"https://edition.cnn.com/search/?size=10&q={q}&page="
  # 아래 range를 변형해서 더 많은 데이터를 수집할 수 있음
  for i in range(1, 4):
    # 처음일 때와 아닐 때 url 값이 바뀌어서 flag를 이용해 변환
    if i == 1:
      url = url + "&page=" + str(i)
    else:
      # 첫 페이지가 아닐 때는 뒤에 &from= (10 * (current_page - 1)) 이 붙음
      url = url.split("&page=")[0]
      url = url + "&page=" + str(i) + f"&from={10 * (i-1)}"
    driver.get(url)
    page_source = driver.page_source
    print(url)
    soup = BeautifulSoup(page_source, 'lxml')
    div_list = soup.findAll("div", {"class":"cnn-search__result-contents"})
    for d in div_list:
      try:
        # 제목, 링크, 콘텐츠, 날짜 값 저장
        href = d.find("h3", {"class":"cnn-search__result-headline"}).find('a').attrs['href']
        title = d.find("h3", {"class":"cnn-search__result-headline"}).find('a').text.strip()
        date = d.find("div", {"class" : "cnn-search__result-publish-date"}).findAll('span')[1].text.strip()
        content = d.find("div", {"class":"cnn-search__result-body"}).text.strip()
        href_list.append(href)
        title_list.append(title)
        date_list.append(date)
        content_list.append(content)
      except:
        continue
    url = url[:-1]
    # 약간의 시간차를 둠
    time.sleep(2)
  show_data(href_list, title_list, date_list, content_list)
  print(f"{q} end")
  # end of driver
  driver.close()
  


In [ ]:
jobs = []
for query in search_list:
    proc = Process(target=crawling, args=(query, driver))
    jobs.append(proc)
    proc.start()
# 모든 multi가 돌아갈 때까지 기다릴 수 있도록 join
for p in jobs:
    p.join()